In [11]:
import xpress as xp
import numpy as np
import pandas as pd
import time

In [12]:
print("="*70)
print("Loading Data")
print("="*70)

Loading Data


In [13]:
# Load distance matrix
distance_matrix = np.loadtxt('distance_matrix.csv', delimiter=',')
stations = pd.read_csv('station_data.csv')
pois = pd.read_csv('edinburgh_pois.csv')

In [14]:
I = distance_matrix.shape[0]  # Number of POIs (demand points)
J = distance_matrix.shape[1] # Number of candidate stations

In [15]:
print("="*70)
print("DEFINE MODEL PARAMETERS")
print("="*70)

DEFINE MODEL PARAMETERS


In [30]:
# IMPORTANT: Set the number of facilities to locate
p = 25  # Change this to test different scenarios

print(f"\n{'='*70}")
print(f"MODEL PARAMETERS")
print(f"{'='*70}")
print(f"Number of POIs (I):                {I}")
print(f"Number of candidate stations (J):  {J}")
print(f"Number of stations to locate (p):  {p}")


MODEL PARAMETERS
Number of POIs (I):                33669
Number of candidate stations (J):  85
Number of stations to locate (p):  25


In [17]:
print("="*70)
print("CREATE XPRESS PROBLEM")
print("="*70)

CREATE XPRESS PROBLEM


In [18]:
prob = xp.problem(name='p_centre_bike_stations')

In [19]:
print("="*70)
print("DEFINE DECISION VARIABLES")
print("="*70)

DEFINE DECISION VARIABLES


In [20]:
# Y[j] = 1 if facility j is opened, 0 otherwise
Y = {j: prob.addVariable(vartype=xp.binary, name=f'Y_{j}') for j in range(J)}

# X[i,j] = 1 if demand point i is assigned to station j, 0 otherwise
X = {(i, j): prob.addVariable(vartype=xp.binary, name=f'X_{i}_{j}') 
     for i in range(I) for j in range(J)}

# Q = maximum service distance (minimax objective)
Q = prob.addVariable(name='Q', lb=0)

print(f"Created {J} Y variables (station selection)")
print(f" Created {I*J:,} X variables (POI-station assignments)")
print(f" Created 1 W variable (minimax distance)")
print(f"  Total variables: {J + I*J + 1:,}")


Created 85 Y variables (station selection)
 Created 2,861,865 X variables (POI-station assignments)
 Created 1 W variable (minimax distance)
  Total variables: 2,861,951


In [21]:
print("="*70)
print("ADD CONSTRAINTS")
print("="*70)

ADD CONSTRAINTS


In [22]:
# CONSTRAINT 1: Each POI must be assigned to exactly one station
print("  Adding constraint 1: POI assignment (each POI assigned to exactly one station)...")
for i in range(I):
    prob.addConstraint(xp.Sum(X[i, j] for j in range(J)) == 1)

# CONSTRAINT 2: Exactly p stations must be opened
print(f"  Adding constraint 2: Facility count (open exactly {p} stations)...")
prob.addConstraint(xp.Sum(Y[j] for j in range(J)) == p)

# CONSTRAINT 3: Assignment only to open stations (linking constraints)
print("  Adding constraint 3: Linking constraints (assign only to open stations)...")
for i in range(I):
    for j in range(J):
        prob.addConstraint(X[i, j] <= Y[j])

# CONSTRAINT 4: Minimax distance constraint
print("  Adding constraint 4: Minimax distance (Q >= distance to assigned station)...")
for i in range(I):
    prob.addConstraint(
        xp.Sum(distance_matrix[i, j] * X[i, j] for j in range(J)) <= Q)

  Adding constraint 1: POI assignment (each POI assigned to exactly one station)...
  Adding constraint 2: Facility count (open exactly 85 stations)...
  Adding constraint 3: Linking constraints (assign only to open stations)...
  Adding constraint 4: Minimax distance (Q >= distance to assigned station)...


In [23]:
print("="*70)
print("SET OBJECTIVE FUNCTION")
print("="*70)

SET OBJECTIVE FUNCTION


In [24]:
# Minimize W (the maximum service distance)
prob.setObjective(Q, sense=xp.minimize)

In [25]:
# prob.write("problem","lp")

In [26]:
print("\n" + "="*70)
print("SOLVING MODEL...")
print("="*70 + "\n")

start_solve = time.time()

# Solve the problem
xp.setOutputEnabled(True)
prob.solve()

solve_time = time.time() - start_solve


SOLVING MODEL...

FICO Xpress v9.7.0, Hyper, solve started 22:15:17, Nov 6, 2025
Heap usage: 1762MB (peak 1762MB, 573MB system)
Minimizing MILP p_centre_bike_stations using up to 8 threads and up to 8192MB memory, with these control settings:
OUTPUTLOG = 1
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = 1
Original problem has:
   2929204 rows      2861951 cols     11481214 elements   2861950 entities
Presolved problem has:
         0 rows            0 cols            0 elements         0 entities
LP relaxation tightened
Presolve finished in 12 seconds
Heap usage: 2606MB (peak 4508MB, 573MB system)
Will try to keep branch and bound tree memory usage below 7.5GB
Starting concurrent solve with dual (1 thread)

 Concurrent-Solve,  12s
            Dual        
    objective   dual inf
 D  7.5562980   .0000000
                        
------- optimal --------
Concurrent statistics:
           Dual: 0 simplex iterations, 0.00s
Optimal solution found
 
   Its         Obj Value      

In [27]:
    print("\n" + "="*70)
    print("SOLUTION SUMMARY")
    print("="*70)
    

    # Extract optimal minimax distance
    Q_optimal = prob.getSolution(Q)
    
    # Extract which stations to open
    stations_open = []
    for j in range(J):
        if prob.getSolution(Y[j]) > 0.5:  # Binary variable, so > 0.5 means 1
            stations_open.append(j)
    



SOLUTION SUMMARY


In [29]:
    print(f"\n{'='*70}")
    print(f"OPTIMAL P-CENTRE RESULTS")
    print(f"{'='*70}")
    
    print(f"\n1. STATIONS TO OPEN ({len(stations_open)} out of {J}):")
    print(f"   Station indices: {stations_open}")
    
    print(f"\n   Station details:")
    optimal_stations_list = []
    for idx, j in enumerate(stations_open, 1):
        station_id = stations.iloc[j]['candidate_id']
        station_name = stations.iloc[j]['name']
        capacity = stations.iloc[j]['capacity']
        lat = stations.iloc[j]['lat']
        lon = stations.iloc[j]['lon']
        
        optimal_stations_list.append({
            'index': j,
            'rank': idx,
            'station_id': station_id,
            'station_name': station_name,
            'capacity': capacity,
            'latitude': lat,
            'longitude': lon
        })
        
        print(f"   {idx:2d}. [{j:2d}] {station_name:30s} (ID: {station_id}, Capacity: {capacity:2d} docks)")
    
    print(f"\n2. OPTIMAL MINIMAX DISTANCE:")
    print(f"   Maximum service distance (W*) = {Q_optimal:.4f} km")
    print(f"   ✓ All {I:,} POIs served within {Q_optimal:.4f} km of nearest station")


OPTIMAL P-CENTRE RESULTS

1. STATIONS TO OPEN (85 out of 85):
   Station indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84]

   Station details:
    1. [ 0] Picardy Place                  (ID: 2268, Capacity: 31 docks)
    2. [ 1] Musselburgh Brunton Hall       (ID: 2265, Capacity: 29 docks)
    3. [ 2] Musselburgh Lidl               (ID: 2263, Capacity: 34 docks)
    4. [ 3] Leith Walk North               (ID: 2259, Capacity: 32 docks)
    5. [ 4] Duke Street                    (ID: 1824, Capacity: 43 docks)
    6. [ 5] Boroughmuir                    (ID: 1823, Capacity: 25 docks)
    7. [ 6] Edinburgh Park Station         (ID: 1822, Capacity: 23 docks)
    8. [ 7] Drumsheugh Place               (ID:

In [20]:
    assignments = {}
    for i in range(I):
        for j in range(J):
            if prob.getSolution(X[i, j]) > 0.5:
                assignments[i] = j
                break

KeyboardInterrupt: 

In [30]:
    print(f"   All {I:,} POIs assigned: {len(assignments) == I}")
    
    # Count POIs per station
    pois_per_station = {}
    distances_per_station = {}
    
    for i, j in assignments.items():
        if j not in pois_per_station:
            pois_per_station[j] = 0
            distances_per_station[j] = []
        pois_per_station[j] += 1
        distances_per_station[j].append(distance_matrix[i, j])
    
    print(f"\n   POIs assigned per station:")
    for rank, j in enumerate(stations_open, 1):
        count = pois_per_station.get(j, 0)
        avg_dist = np.mean(distances_per_station.get(j, ))
        max_dist = np.max(distances_per_station.get(j, ))
        
        print(f"   {rank:2d}. {stations.iloc[j]['name']:30s}: {count:5,} POIs (avg: {avg_dist:.3f} km, max: {max_dist:.3f} km)")
    

NameError: name 'assignments' is not defined

In [31]:
    all_assigned_distances = [distance_matrix[i, assignments[i]] for i in range(I)]
    
    print(f"\n4. DISTANCE STATISTICS:")
    print(f"   Mean distance: {np.mean(all_assigned_distances):.4f} km")
    print(f"   Median distance: {np.median(all_assigned_distances):.4f} km")
    print(f"   Std Dev: {np.std(all_assigned_distances):.4f} km")
    print(f"   Min distance: {np.min(all_assigned_distances):.4f} km")
    print(f"   Max distance (W*): {np.max(all_assigned_distances):.4f} km")
    
    # Coverage analysis
    print(f"\n5. COVERAGE ANALYSIS:")
    print(f"   POIs within 0.5 km: {(np.array(all_assigned_distances) < 0.5).sum():,} ({100*(np.array(all_assigned_distances) < 0.5).sum()/I:.2f}%)")
    print(f"   POIs within 1.0 km: {(np.array(all_assigned_distances) < 1.0).sum():,} ({100*(np.array(all_assigned_distances) < 1.0).sum()/I:.2f}%)")
    print(f"   POIs within 2.0 km: {(np.array(all_assigned_distances) < 2.0).sum():,} ({100*(np.array(all_assigned_distances) < 2.0).sum()/I:.2f}%)")
    

NameError: name 'assignments' is not defined

In [20]:
demand_by_category = {
    'residential': 100,
    'commercial': 60,
    'school': 50,
    'university': 80,
    'library': 30,
    'hospital': 40
}

demand = np.array([
    demand_by_category.get(cat, 50)
    for cat in pois['category']
])



In [21]:
demand

array([ 30,  50,  30, ..., 100, 100, 100], shape=(33669,))